In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [ ]:
instructions1 = "Eres un agente de ventas que trabaja para ComplAI, \
    una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2\
    y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos en frío profesionales y serios."

instructions2 = "Eres un agente de ventas con sentido del humor y atractivo \
    que trabaja para ComplAI, una empresa que ofrece una herramienta SaaS para \
    garantizar el cumplimiento de SOC2 y prepararse para auditorías, impulsada por IA. \
    Redactas correos electrónicos en frío ingeniosos y atractivos que probablemente obtengan respuesta."

instructions3 = "Eres un agente de ventas muy activo que trabaja para ComplAI, \
    una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2\
    y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos en frío concisos y directos."